In [90]:
import sourmash
import screed
import os

from tqdm import tqdm

from kmer_utils import get_encoded_kmer_hashvals

In [1]:
cd  ~/botryllus/pipeline-results/kmermaid/april2021-singleton-fewer-species/sketches_peptide/molecule-protein,dayhoff,hp__ksize-30,51,72,81__scaled-10__track_abundance-true/sigs 


/Users/olgabot/botryllus/pipeline-results/kmermaid/april2021-singleton-fewer-species/sketches_peptide/molecule-protein,dayhoff,hp__ksize-30,51,72,81__scaled-10__track_abundance-true/sigs


In [2]:
ls

Arabidopsis_thaliana.TAIR10.pep.all.fa__molecule-protein,dayhoff,hp__ksize-30,51,72,81__scaled-10__track_abundance-true.sig
Bs_proteins.fa__molecule-protein,dayhoff,hp__ksize-30,51,72,81__scaled-10__track_abundance-true.sig
Caenorhabditis_elegans.WBcel235.pep.all.fa__molecule-protein,dayhoff,hp__ksize-30,51,72,81__scaled-10__track_abundance-true.sig
Danio_rerio.GRCz11.pep.abinitio.fa__molecule-protein,dayhoff,hp__ksize-30,51,72,81__scaled-10__track_abundance-true.sig
Danio_rerio.GRCz11.pep.all.fa__molecule-protein,dayhoff,hp__ksize-30,51,72,81__scaled-10__track_abundance-true.sig
Homo_sapiens.GRCh38.pep.all.chr6.fa__molecule-protein,dayhoff,hp__ksize-30,51,72,81__scaled-10__track_abundance-true.sig
Mus_musculus.GRCm39.pep.all.chr17.fa__molecule-protein,dayhoff,hp__ksize-30,51,72,81__scaled-10__track_abundance-true.sig
Petromyzon_marinus.Pmarinus_7.0.pep.abinitio.fa__molecule-protein,dayhoff,hp__ksize-30,51,72,81__scaled-10__track_abundance-true.sig
botryllus_bhf.sig
human_mouse/


In [37]:
human_gencode_dir = '/Users/olgabot/botryllus/data/gencode/v38/'
mouse_gencode_dir = '/Users/olgabot/botryllus/data/gencode/M28/'

In [41]:
human_sigfile = os.path.join(
    human_gencode_dir, "gencode.v38.basic.annotation.protein.fa.hp.k24.scale5.sig"
)
mouse_sigfile = os.path.join(
    mouse_gencode_dir, "gencode.vM28.basic.annotation.protein.fa.hp.k24.scale5.sig"
)

## Assign outdir

In [42]:
outdir = '/Users/olgabot/botryllus/adhoc-analysis/2022-jan--redo-human-mouse-search-gencode'

In [43]:
! sourmash search --hp --threshold 0.01 \
    -o $outdir/human_search_mouse.csv $human_sigfile $mouse_sigfile


== This is sourmash version 4.2.2. ==
== Please cite Brown and Irber (2016), doi:10.21105/joss.00027. ==



## Load and index human signatures, 

In [44]:
human_sigfile_hp_k24 = 'Homo_sapiens.GRCh38.pep.all.chr6__hp.k24.scaled10.sig'

In [47]:
# ! sourmash sig extract -o $human_sigfile_hp_k24 --hp -k 24 $human_sigfile

In [48]:
human_index = sourmash.load_file_as_index(human_sigfile)
human_index

## Read mouse signatures

In [49]:
mouse_sigs = sourmash.load_file_as_signatures(mouse_sigfile, select_moltype='hp', ksize=24)

## Load mouse and human protein fasta files to get the underlying overlapping k-mers

In [50]:
human_fasta = os.path.join(human_gencode_dir, "gencode.v38.basic.annotation.protein.fa")
mouse_fasta = os.path.join(
    mouse_gencode_dir, "gencode.vM28.basic.annotation.protein.fa"
)

In [51]:
mouse_seqs = {record['name']: record['sequence'] for record in screed.open(mouse_fasta)}
human_seqs = {record['name']: record['sequence'] for record in screed.open(human_fasta)}

In [52]:
len(mouse_seqs)

46083

In [53]:
len(human_seqs)

61543

# Iterate over mouse signatures

In [89]:
mouse_sig.minhash.seq_to_hashes?

Signature:
mouse_sig.minhash.seq_to_hashes(
    sequence,
    *,
    force=False,
    bad_kmers_as_zeroes=False,
    is_protein=False,
)
Docstring:
Convert sequence to hashes without adding to the sketch.

If input sequence is DNA and this is a protein, dayhoff, or hp
MinHash, translate the DNA appropriately before hashing.

If input sequence is protein, set is_protein=True.

If `force = True` and `bad_kmers_as_zeroes = True`,
invalid kmers hashes will be represented as `0`.
File:      ~/opt/miniconda3/envs/botryllus/lib/python3.10/site-packages/sourmash/minhash.py
Type:      method


In [92]:
%time

overlapping_kmer_dfs = []
similarity_lines = []

for mouse_sig in tqdm(mouse_sigs):
    results = human_index.search_abund(mouse_sig, threshold=0.01)
    mouse_seq = mouse_seqs[mouse_sig.name]
    mouse_kmers = get_encoded_kmer_hashvals(mouse_seq, mouse_sig.name, sigobj=mouse_sig)
    for result in results:
        # Add a line to the similarity dataframe
        similarity_line = [mouse_sig.name, result.score, result.signature.name]
        similarity_lines.append(similarity_line)
        
        # Get overlapping k-mers
        human_seq = human_seqs[result.signature.name]
        human_kmers = get_encoded_kmer_hashvals(human_seq, mouse_sig.name, sigobj=mouse_sig)
        overlapping_kmers = human_kmers.merge(mouse_kmers, on=['kmer', 'hashval'], suffixes=('_human', '_mouse'))
        overlapping_kmer_dfs.append(overlapping_kmers)

similarity_results = pd.DataFrame(similarity_lines, columns=['mouse_gencode', 'similarity', 'human_gencode'])
print(similarity_results.shape)
similarity_results.head()

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.96 µs


46077it [72:56:51,  5.70s/it]   


NameError: name 'pd' is not defined

In [ ]:
overlapping_kmer_results = pd.concat(overlapping_kmer_dfs, ignore_index=True)
print(overlapping_kmer_results.shape)
overlapping_kmer_results.head()

In [86]:
results[0]

Result(score=0.07060555345730624, signature=SourmashSignature('ENST00000297316.5 gene_id=ENSG00000164736.6;transcript_id=ENST00000297316.5;gene_type=protein_coding;gene_name=SOX17;transcript_type=protein_coding;transcript_name=SOX17-201;level=2;protein_id=ENSP00000297316.4;transcript_support_level=1;hgnc_id=HGNC:18122;tag=basic,Ensembl_canonical,MANE_Select,appris_principal_1,CCDS;ccdsid=CCDS6159.1;havana_gene=OTTHUMG00000164377.3;havana_transcript=OTTHUMT00000378526.3', 8b23329d), location=None)

In [87]:
result = results[0]

In [88]:
result.

Result(score=0.07060555345730624, signature=SourmashSignature('ENST00000297316.5 gene_id=ENSG00000164736.6;transcript_id=ENST00000297316.5;gene_type=protein_coding;gene_name=SOX17;transcript_type=protein_coding;transcript_name=SOX17-201;level=2;protein_id=ENSP00000297316.4;transcript_support_level=1;hgnc_id=HGNC:18122;tag=basic,Ensembl_canonical,MANE_Select,appris_principal_1,CCDS;ccdsid=CCDS6159.1;havana_gene=OTTHUMG00000164377.3;havana_transcript=OTTHUMT00000378526.3', 8b23329d), location=None)

In [82]:
human_kmers.head()

,i,kmer,kmer_hp,hashval,name
0,0,MSSPDAGYASDDQSQTQSALPAVM,hpphphhhhppppppppphhhhhh,12803979974930755435,ENSMUST00000195555.2 gene_id=ENSMUSG0000002590...
1,1,SSPDAGYASDDQSQTQSALPAVMA,pphphhhhppppppppphhhhhhh,7379593282333811979,ENSMUST00000195555.2 gene_id=ENSMUSG0000002590...
2,2,SPDAGYASDDQSQTQSALPAVMAG,phphhhhppppppppphhhhhhhh,5799147356561468663,ENSMUST00000195555.2 gene_id=ENSMUSG0000002590...
3,3,PDAGYASDDQSQTQSALPAVMAGL,hphhhhppppppppphhhhhhhhh,13507419260536542374,ENSMUST00000195555.2 gene_id=ENSMUSG0000002590...
4,4,DAGYASDDQSQTQSALPAVMAGLG,phhhhppppppppphhhhhhhhhh,6604397787272783784,ENSMUST00000195555.2 gene_id=ENSMUSG0000002590...


,i_human,kmer,kmer_hp_human,hashval,name_human,i_mouse,kmer_hp_mouse,name_mouse
0,389,DSHGAISSVVSDASSAVYYCNYPD,ppphhhpphhpphpphhhhpphhp,4488222948142703667,ENSMUST00000195555.2 gene_id=ENSMUSG0000002590...,266,ppphhhpphhpphpphhhhpphhp,ENSMUST00000195555.2 gene_id=ENSMUSG0000002590...


In [77]:
mouse_kmers

,i,kmer,kmer_hp,hashval,name
0,0,MQDHPNYKYRPRRRKQVKRMKRVE,hppphphphphppppphpphpphp,4675538599979812702,ENSMUST00000195555.2 gene_id=ENSMUSG0000002590...
1,1,QDHPNYKYRPRRRKQVKRMKRVEG,ppphphphphppppphpphpphph,16787700624938120076,ENSMUST00000195555.2 gene_id=ENSMUSG0000002590...
2,2,DHPNYKYRPRRRKQVKRMKRVEGG,pphphphphppppphpphpphphh,10958952744925319120,ENSMUST00000195555.2 gene_id=ENSMUSG0000002590...
3,3,HPNYKYRPRRRKQVKRMKRVEGGF,phphphphppppphpphpphphhh,11073673144508362015,ENSMUST00000195555.2 gene_id=ENSMUSG0000002590...
4,4,PNYKYRPRRRKQVKRMKRVEGGFL,hphphphppppphpphpphphhhh,3458521422108452682,ENSMUST00000195555.2 gene_id=ENSMUSG0000002590...
...,...,...,...,...,...
263,263,NLSDSHGAISSVVSDASSAVYYCN,phpppphhhpphhpphpphhhhpp,4800405316494767242,ENSMUST00000195555.2 gene_id=ENSMUSG0000002590...
264,264,LSDSHGAISSVVSDASSAVYYCNY,hpppphhhpphhpphpphhhhpph,12758218761529981668,ENSMUST00000195555.2 gene_id=ENSMUSG0000002590...
265,265,SDSHGAISSVVSDASSAVYYCNYP,pppphhhpphhpphpphhhhpphh,15172368290909897604,ENSMUST00000195555.2 gene_id=ENSMUSG0000002590...
266,266,DSHGAISSVVSDASSAVYYCNYPD,ppphhhpphhpphpphhhhpphhp,4488222948142703667,ENSMUST00000195555.2 gene_id=ENSMUSG0000002590...


In [70]:
mouse_seq = mouse_seqs[mouse_sig.name]
mouse_seq

'MSSPDAGYASDDQSQPRSAQPAVMAGLGPCPWAESLSPLGDVKVKGEVVASSGAPAGTSGRAKAESRIRRPMNAFMVWAKDERKRLAQQNPDLHNAELSKMLGKSWKALTLAEKRPFVEEAERLRVQHMQDHPNYKYRPRRRKQVKRMKRVEGGFLHALVEPQAGALGPEGGRVAMDGLGLPFPEPGYPAGPPLMSPHMGPHYRDCQGLGAPALDGYPLPTPDTSPLDGVEQDPAFFAAPLPGDCPAAGTYTYAPVSDYAVSVEPPAGPMRVGPDPSGPAMPGILAPPSALHLYYGAMGSPAASAGRGFHAQPQQPLQPQAPPPPPQQQHPAHGPGQPSPPPEALPCRDGTESNQPTELLGEVDRTEFEQYLPFVYKPEMGLPYQGHDCGVNLSDSHGAISSVVSDASSAVYYCNYPDI'

In [71]:
mouse_sig.name

'ENSMUST00000027035.10 gene_id=ENSMUSG00000025902.14;transcript_id=ENSMUST00000027035.10;gene_type=protein_coding;gene_name=Sox17;transcript_type=protein_coding;transcript_name=Sox17-201;level=2;protein_id=ENSMUSP00000027035.4;transcript_support_level=1;mgi_id=MGI:107543;tag=basic,appris_principal_1,CCDS;ccdsid=CCDS14805.1;havana_gene=OTTMUSG00000050014.7;havana_transcript=OTTMUST00000127245.2'

In [72]:
results


[Result(score=0.17478066219902832, signature=SourmashSignature('ENST00000297316.5 gene_id=ENSG00000164736.6;transcript_id=ENST00000297316.5;gene_type=protein_coding;gene_name=SOX17;transcript_type=protein_coding;transcript_name=SOX17-201;level=2;protein_id=ENSP00000297316.4;transcript_support_level=1;hgnc_id=HGNC:18122;tag=basic,Ensembl_canonical,MANE_Select,appris_principal_1,CCDS;ccdsid=CCDS6159.1;havana_gene=OTTHUMG00000164377.3;havana_transcript=OTTHUMT00000378526.3', 8b23329d), location=None),
 Result(score=0.05203771602417573, signature=SourmashSignature('ENST00000527137.2 gene_id=ENSG00000090581.11;transcript_id=ENST00000527137.2;gene_type=protein_coding;gene_name=GNPTG;transcript_type=protein_coding;transcript_name=GNPTG-204;level=2;protein_id=ENSP00000480060.1;transcript_support_level=2;hgnc_id=HGNC:23026;tag=basic;havana_gene=OTTHUMG00000047835.7;havana_transcript=OTTHUMT00000395320.2', 01fa4f12), location=None),
 Result(score=0.049991927179489415, signature=SourmashSignature